In [ ]:
import warnings
warnings.filterwarnings('ignore')



import pandas as pd
import numpy as np



import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16,9]
%matplotlib inline
import seaborn as sns
import os



from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from datetime import timedelta, date, datetime



import statsmodels.api as sm



import pmdarima as pm
from pmdarima.model_selection import train_test_split



#os.chdir("C://Tarun\Projects_2021/ML_DL_Accelerators/Time_Series")
# data_path = os.path.join(os.getcwd(), "Data/")
MAIN_DIR = '\\'.join(os.getcwd().split('\\')[:-1])
data_path = MAIN_DIR+'\\data\\raw\\store_item\\'



df = pd.read_csv(data_path +'train.csv', parse_dates=['date'])




df['store'] = df['store'].astype('category')
df['item'] = df['item'].astype('category')
df['sales'] = df['sales'].astype('int32')



df_1 = df[(df['store'] == 1) & (df['item'] == 1)]




data = df_1.drop(columns=['store', 'item'], axis = 1)



data = data.groupby(['date']).agg({'sales' : 'sum'})



data.plot()



decompose_data = seasonal_decompose(data, model="additive", freq=365)
#decompose_data = seasonal_decompose(data, model="additive")
decompose_data.plot();




from statsmodels.tsa.stattools import adfuller
dftest = adfuller(data.sales, autolag = 'AIC')
print("1. ADF : ",dftest[0])
print("2. P-Value : ", dftest[1])
print("3. Num Of Lags : ", dftest[2])
print("4. Num Of Observations Used For ADF Regression and Critical Values Calculation :", dftest[3])
print("5. Critical Values :")
for key, val in dftest[4].items():
    print("\t",key, ": ", val)



if dftest[1] > 0.05 :
    print('Series is not Stationary')
else:
    print('Series is Stationary')



#!pip install pmdarima




data.head()








model = pm.auto_arima(
    y = data['sales'],
    m=365,
    seasonal=True,
    start_p=1, start_q=1, max_p=3, max_q=3,
    max_order=4,
    test='adf',
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True,
    trace=True,
    n_jobs = -1
)

1. ADF :  -3.1576705563328114
2. P-Value :  0.02256938062657105
3. Num Of Lags :  23
4. Num Of Observations Used For ADF Regression and Critical Values Calculation : 1802
5. Critical Values :
	 1% :  -3.4339840952648695
	 5% :  -2.8631452508003057
	 10% :  -2.567624583142913
Series is Stationary
Performing stepwise search to minimize aic


In [ ]:

import pickle
with open('temp_arima.pkl') as mf:
    pickle.dump(mf, model)

# %%time



# model = pm.auto_arima(y = data['sales'],
# m=12, seasonal=True,
# start_p=1, start_q=1, max_p=3, max_q=3, max_order=4, test='adf',error_action='ignore',
# suppress_warnings=True,
# stepwise=True, trace=True, n_jobs = -1)





# %%time



# model = pm.auto_arima(y = data['sales'],
# m=1, seasonal=True,
# start_p=1, start_q=1, max_p=3, max_q=3, max_order=4, test='adf',error_action='ignore',
# suppress_warnings=True,
# stepwise=True, trace=True, n_jobs = -1)

In [ ]:
model.summary